In [123]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Input,Dense,concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')

In [124]:
#所有的数据列
COLUMNS = [
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status", 
    "occupation", "relationship", "race", "gender", "capital_gain", "capital_loss", 
    "hours_per_week", "native_country", "income_bracket"
]

#标签列
LABEL_COLUMN = "label"

#类别型特征变量
CATEGORICAL_COLUMNS = [
    "workclass", "education", "marital_status", "occupation", "relationship", 
    "race", "gender", "native_country"
]

#连续值特征变量
CONTINUOUS_COLUMNS = [
    "age", "education_num", "capital_gain", "capital_loss", "hours_per_week"
]

In [125]:
#加载文件
def load(filename):
    with open(filename, 'r') as f:
        skiprows = 1 if 'test' in filename else 0
        df = pd.read_csv(
            f, names=COLUMNS, skipinitialspace=True, skiprows=skiprows, engine='python'
        )
        #缺省值处理
        df = df.dropna(how='any', axis=0)
    return df

In [126]:
#预处理
def preprocess(df):
    df[LABEL_COLUMN] = df['income_bracket'].apply(lambda x: ">50K" in x).astype(int)
    df.pop("income_bracket")
    y = df[LABEL_COLUMN].values
    df.pop(LABEL_COLUMN)
    
    df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])

    # TODO: 对特征进行选择，使得网络更高效
    
    # TODO: 特征工程，比如加入交叉与组合特征
    # from sklearn.preprocessing import PolynomialFeatures
    # X = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False).fit_transform(X)
    
    df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)

    X = df.values
    return X, y

In [127]:
def main():
    df_train = load('adult.data')
    df_test = load('adult.test')
    df = pd.concat([df_train, df_test])
    train_len = len(df_train)
    
    X, y = preprocess(df)
    X_train = X[:train_len]
    y_train = y[:train_len]
    X_test = X[train_len:]
    y_test = y[train_len:]
    
    #Wide部分
    wide_inputs = Input(shape=(X_train.shape[1],))
    wide=Dense(1, activation='relu')(wide_inputs)
    
    #Deep部分
    # TODO: 添加embedding层
    deep_inputs = Input(shape=(X_train.shape[1],))
    deep= Dense(input_dim=X_train.shape[1], output_dim=100, activation='relu')(deep_inputs)
    #deep.add(Dense(100, activation='relu'))
    deep= Dense(input_dim=100, output_dim=32, activation='relu')(deep)
    #deep.add(Dense(50, activation='relu'))
    deep= Dense(input_dim=32, output_dim=8)(deep)
    deep= Dense(1, activation='sigmoid')(deep)
    
    #Wide和Deep拼接
    merge_wide_deep = concatenate([wide, deep])
    predictions=Dense(1, activation='sigmoid')(merge_wide_deep)
    model = Model(inputs=[wide_inputs,deep_inputs], outputs=predictions)
    
    #编译模型
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    #模型训练
    model.fit([X_train, X_train], y_train, nb_epoch=10, batch_size=32)
    
    #loss与准确率评估
    loss, accuracy = model.evaluate([X_test, X_test], y_test)
    print('\n', 'test accuracy:', accuracy)

In [128]:
if __name__ == '__main__':
    main()

Epoch 1/10
32561/32561 [==============================] - 2s 59us/step - loss: 0.4412 - acc: 0.7747
Epoch 2/10
32561/32561 [==============================] - 1s 43us/step - loss: 0.3673 - acc: 0.8367
Epoch 3/10
32561/32561 [==============================] - 1s 44us/step - loss: 0.3513 - acc: 0.8413
Epoch 4/10
32561/32561 [==============================] - 1s 44us/step - loss: 0.3435 - acc: 0.8431
Epoch 5/10
32561/32561 [==============================] - 1s 44us/step - loss: 0.3387 - acc: 0.8451
Epoch 6/10
32561/32561 [==============================] - 1s 45us/step - loss: 0.3355 - acc: 0.8477
Epoch 7/10
32561/32561 [==============================] - 1s 43us/step - loss: 0.3325 - acc: 0.8486
Epoch 8/10
32561/32561 [==============================] - 1s 46us/step - loss: 0.3305 - acc: 0.8498
Epoch 9/10
32561/32561 [==============================] - 1s 46us/step - loss: 0.3285 - acc: 0.8505
Epoch 10/10
16281/16281 [==============================] - 0s 25us/step

 test accuracy: 0.849026472